># Universidad Autónoma de Aguascalientes
>## *Ingeniería en Computación Inteligente*
>#### Materia:
>Inteligencia Artificial
>#### Topico: 
>Exámen Final:
>- _Predicción del uso del coreceptor de tropismos de VIH-1._
>#### Integrantes del Equipo:
>- Juan Francisco Gallo Ramírez
>- José Alfredo Díaz Robledo
>- Luis Palbo Esparza Terrones
>- Luis Manuel Flores Jiménez 
>#### Maestro: 
>Dr. Francisco Javier Luna Rosas
>#### Fecha: 
>Noviembre del 2023

# Exámen Final: _Predicción del uso del coreceptor de tropismos de VIH-1._

## ▪️ Se importan la librerías correspondientes.

In [1]:
import numpy as np
import openpyxl
import pandas as pd
import requests
from bs4 import BeautifulSoup
from colorama import Back, Fore
from selenium import webdriver
from selenium.webdriver.common.by import By
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import time
import random

## ▪️ Se establecen diccionarios de características.

Se establecen los diccionarios de características de los aminoácidos, los cuales son de utilidad para  la realización del DataSet. En este algoritmo se usan las caracteristicas de masa, área de superficie, volumen, etc.

In [2]:
# Masa:
mass = { 'A': 71.09, 'R': 156.19, 'N': 114.11, 'D': 115.09, 'C': 103.15, 'E': 129.12, 'Q': 128.14, 'G': 57.05, 
         'H': 137.14, 'I': 113.16, 'L': 113.16, 'K': 128.17, 'M': 131.19, 'F': 147.18, 'P':  97.12, 'S': 87.08, 
         'T': 101.11, 'W': 186.21, 'Y': 163.18, 'V': 99.14 }

# Área de la superficie:
s_area = { 'A': 0.74 , 'R': 0.64, 'N': 0.63, 'D': 0.62, 'C': 0.91, 'E': 0.62, 'Q': 0.62, 'G': 0.72, 
           'H': 0.78, 'I': 0.88, 'L': 0.85, 'K': 0.52, 'M': 0.85, 'F': 0.88, 'P':  0.64, 'S': 0.66, 
           'T': 0.7, 'W': 0.85, 'Y': 0.76, 'V': 0.86 }

# Volumen:
volume = { 'A': 67, 'R': 148, 'N': 96, 'D': 91, 'C': 86, 'E': 109, 'Q': 114, 'G': 48, 'H': 118, 'I': 124, 
           'L': 124, 'K': 135, 'M': 124, 'F': 135, 'P': 90, 'S': 73, 'T': 93, 'W': 163, 'Y': 141, 'V': 105 }


## ▪️ Se crea archivo del DataSet general.

Se definen las columnas del DataSet, las cuales son 46 en total. Una vez creado en dataframe se procede a guardarlo con el nombre de "DataSet.xlxs".

In [3]:
# Nombre de las columnas del DataSet.
columnas = ['#Ala (A)', '%Ala (A)', '#Arg (R)', '%Arg (R)', '#Asn (N)', '%Asn (N)', '#Asp (D)', '%Asp (D)',
            '#Cys (C)', '%Cys (C)', '#Glu (E)', '%Glu (E)', '#Gln (Q)', '%Gln (Q)', '#Gly (G)', '%Gly (G)',
            '#His (H)', '%His (H)', '#Ile (I)', '%Ile (I)', '#Leu (L)', '%Leu (L)', '#Lys (K)', '%Lys (K)',
            '#Met (M)', '%Met (M)', '#Phe (F)', '%Phe (F)', '#Pro (P)', '%Pro (P)', '#Ser (S)', '%Ser (S)', 
            '#Thr (T)', '%Thr (T)', '#Trp (W)', '%Trp (W)', '#Tyr (Y)', '%Tyr (Y)', '#Val (V)', '%Val (V)',
            '# Aminoacid', 'Volume', 'Mass', 'Surface Area', 'Isoelectric point', 'Net charge at pH 7.4', 'Correceptor']
# Se crea el data frame y se guarda.
ds = openpyxl.Workbook()
sheet = ds.active
sheet.append(columnas)
ds.save('DataSet.xlsx')

## ▪️ Se carga el archivo con los identificadores

Se carga el archivo con todos los identificadores de tropismos R5, X4 y R5X4 y se imprimen.

In [4]:
data = pd.read_excel('NumeroAccesoSecuencias.xlsx')
print(Back.WHITE  + " >>> Números de acceso para las secuencias <<< \n" + Back.RESET + Fore.RESET)
print(data)

 >>> Números de acceso para las secuencias <<< 

        R5X4        R5        X4
0   AB014795  AF062012  AB014785
1   AF062029    L03698  AB014791
2   AF062031  AF231045  AB014796
3   AF062033  AY669778  AB014810
4   AF107771    U08810    U48267
..       ...       ...       ...
72       NaN  AF355326       NaN
73       NaN    U88826       NaN
74       NaN    U08368       NaN
75       NaN    U27426       NaN
76       NaN  AJ006022       NaN

[77 rows x 3 columns]


## ▪️ Se define la función de búsqueda de datos.

Esta función es la encargada de relizar el web scraping para la obtención de la cadena de aminoácidos, así como la extracción de más información de dicha cadena. La función retornará la fila para el DataSet.

In [5]:
def getIdenData(identificador):
    
    #=== Se obtiene la cadena de aminoácidos según el identificador ===============
    fila = []
    com = True
    while com:
        com = False;
        driver = webdriver.Chrome()
        # Página para obtención de la cadena de aminoácidos.
        driver.get('https://www.ncbi.nlm.nih.gov/nuccore/' + identificador)
        time.sleep(3)
        html_content = driver.page_source
        
        soup = BeautifulSoup(html_content, 'html.parser')
        spans = soup.find_all('span', class_='ff_line')
        origen = ""
        origen = "".join([span.text for span in spans])
        origen = origen.replace(' ', '').replace('\n', '')
        if len(origen) == 0:
            com = True
            driver.close()

    # Se extraen las propiedades de la cadena de aminoácidos.
    total = len(origen)
    cant_amin = { 
        'A': 0, 'R': 0, 'N': 0, 'D': 0, 'C': 0, 'E': 0, 'Q': 0, 'G': 0, 'H': 0, 'I': 0, 
        'L': 0, 'K': 0, 'M': 0, 'F': 0, 'P': 0, 'S': 0, 'T': 0, 'W': 0, 'Y': 0, 'V': 0 }
    vol = 0.0
    mas = 0.0
    sur = 0.0
    for letra in origen:
        # Se suma la cantidad del aminoácido.
        cant_amin[letra.upper()] += 1
        # Se suma el volumen del aminoácido.
        vol += volume[letra.upper()]
        # Se suma la masa del aminoácido.
        mas += mass[letra.upper()]
        # Se suma el área de superficie del aminoácido.
        sur += s_area[letra.upper()]

    # Se agregan los datos a la fila.
    for cant in cant_amin.values():
        fila.append(float(cant))
        fila.append(float(cant*100/total))
    fila.append(total)
    fila.append(vol)
    fila.append(mas)
    fila.append(sur)

    #=== Se extraen otras propiedades de la cadena de aminoácidos =================
    com = True
    while com:
        com = False;
        driver = webdriver.Chrome()
        # Página para obtención de más características.
        driver.get('https://www.protpi.ch/Calculator/ProteinTool')
        textarea = driver.find_element(By.ID, 'Sequences_0__Sequence')
        textarea.send_keys(origen)
        boton_enviar = driver.find_element(By.ID, 'submit')
        boton_enviar.click()
        time.sleep(2)
        html_content = driver.page_source
        driver.close()

        if len(html_content) == 0:
            com = True
            driver.close()
        else:
            soup = BeautifulSoup(html_content, 'html.parser')
             # Se agregan los datos a la fila.
            fila.extend([float(strong.text) 
                         for strong in soup.select('div#isoelectricPointCollapse div.card-header div.col-sm-6 strong')])
    driver.quit()
    
    # Se retorna la fila con datos.  
    return fila

## ▪️ Web Scraping para indentificadores R5.

Se recorre la columna con los identificadores del tipo R5, y en base a su identificador, se usa la función "getIdenData()" para  obtener la fila de información para el DataSet.

In [6]:
print(Back.WHITE  + " >>> IDENTIFICADORES COMPLETADOS: <<< \n" + Back.RESET + Fore.RESET)

# Se recorren todos los identificadores "R5".
for iden in data['R5'].dropna():
    
    # Se obtienen los datos del identificador y se agregan al data frame.
    fila = getIdenData(iden)
    fila.append('R5')
    sheet.append(fila)
    print(iden, end = " ")
    
# Se guarda el data frame en el archivo excel.
ds.save('DataSet.xlsx') 
print(Fore.GREEN  + "\n\n+++ PROCESO COMPLETADO +++" + Fore.RESET)

 >>> IDENTIFICADORES COMPLETADOS: <<< 

AF062012 L03698 AF231045 AY669778 U08810 U51296 AF407161 AB253421 U08645 U08647 U08795 AB253429 AY288084 AF307753 AF411964 U08823 AF411965 U92051 AF355318 AY010759 AY010804 AY010852 U08670 U08798 AY669715 U08710 U16217 M26727 AJ418532 AJ418479 AJ418495 AJ418514 AJ418521 U23487 U04900 AF022258 AF258957 AF021477 ﻿U08716 U39259 AF204137 M38429 U27443 U79719 U04909 U04918 U04908 U08450 AF112542 M63929 U66221 AF491737 U08779 L22084 U27413 AF005495 U52953 AF321523 L22940 U45485 AB023804 U08453 AF307755 AF307750 AY043176 AY158534 AX455917 AY043173 AF307757 U08803 U88824 U69657 AF355326 U88826 U08368 U27426 AJ006022 

+++ PROCESO COMPLETADO +++


## ▪️ Web Scraping para indentificadores X4.

Se recorre la columna con los identificadores del tipo X4, y en base a su identificador, se usa la función "getIdenData()" para  obtener la fila de información para el DataSet.

In [7]:
print(Back.WHITE  + " >>> IDENTIFICADORES COMPLETADOS: <<< \n" + Back.RESET + Fore.RESET)

# Se recorren todos los identificadores "X4".
for iden in data['X4'].dropna():
    
    # Se obtienen los datos del identificador y se agregan al data frame.
    fila = getIdenData(iden)
    fila.append('X4')
    sheet.append(fila)
    print(iden, end = " ")
    
# Se guarda el data frame en el archivo excel.
ds.save('DataSet.xlsx') 
print(Fore.GREEN  + "\n\n+++ PROCESO COMPLETADO +++" + Fore.RESET)

 >>> IDENTIFICADORES COMPLETADOS: <<< 

AB014785 AB014791 AB014796 AB014810 U48267 U08666 AF069672 AF355319 AF355336 M14100 A04321 X01762 L31963 U08447 AF355660 AF355748 AF355742 AF355706 AF180915 AF180903 AF035534 AF259050 AF258981 AF259003 AF021618 AF128989 M17449 AF075720 U48207 U72495 AY189526 AF034375 AF034376 U27408 AF411966 U27399 U08822 U08738 U08740 U08193 AF355330 

+++ PROCESO COMPLETADO +++


## ▪️ Obtención de las columnas óptimas.

Función para calcular los índices de calidad de la predicción:

In [8]:
def indices_general(MC, nombres = None):
    precision_global = np.sum(MC.diagonal()) / np.sum(MC)
    error_global = 1 - precision_global
    return {"> Matriz de Confusión":MC, 
            "> Precisión Global":precision_global, 
            "> Error Global":error_global}

Este fragmento de código es el encargado de hacer las combinaciones aleatorias para la obtención de ina precisión  superior al 90%. En base a las columnas del DataSet general obtenido en las celdas anteriores se hace una combinación de columnas aleatorias y se entrena la red neuronal con dichas columnas, si no se cumple la condición de precisión de más de 90%, se realiza otra combinación.

In [9]:
# Se carga el DataSet general.
data = pd.read_excel('DataSet.xlsx')
precision_global = 0

# Condición de precisión.
while precision_global < 0.9:
    # Seleccionar aleatoriamente un número de columnas del DataSet.
    cant = random.randint(1, 46)
    # Evitar repetir columnas seleccionadas.
    col = random.sample(range(46), cant)
    X = data.iloc[:, col]
    y = data.iloc[:, 46]
    caracteristicas = X
    categoria = y
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)
    # Se entrena la red.
    instancia_red = MLPClassifier(solver='adam', random_state=0, max_iter=2000)
    instancia_red.fit(X_train, y_train)
    prediccion = instancia_red.predict(X_test)
    MC = confusion_matrix(y_test, prediccion)
    precision_global = np.sum(np.diag(MC)) / np.sum(MC)

# Calcular e imprimir los índices generales y las características adecuadas.
print(Back.WHITE  + " >>> CARACTERÍSTICAS ÓPTIMAS <<< \n" + Back.RESET)
print(caracteristicas.columns.values)
indices = indices_general(MC, list(np.unique(y)))
print(Back.YELLOW + "\n - Índices obtenidos: " + Back.RESET + Fore.RESET)
for k in indices:
    print("\n%s:\n%s" % (k, str(indices[k])))

nuevo = pd.concat([caracteristicas, categoria], axis=1)
nuevo.to_excel('DataSetOptimo.xlsx', index=False)

 >>> CARACTERÍSTICAS ÓPTIMAS <<< 

['#Ala (A)' '#Trp (W)' '%Ala (A)' '#Phe (F)' '%Asn (N)']

 - Índices obtenidos: 

> Matriz de Confusión:
[[25  0]
 [ 3  8]]

> Precisión Global:
0.9166666666666666

> Error Global:
0.08333333333333337


## ▪️ Red Neuronal.

Se lee el archivo del DataSet óptimo, y se muestran la respectiuva matriz categórica y matriz a predecir.

In [10]:
data = pd.read_excel('DataSetOptimo.xlsx')
X = data.iloc[:,0:5]
y = data.iloc[:,5]
X = np.array(X)
y = np.array(y)
print( ">> Se muestra la matriz categorica:\n")
print(X)
print( "\n>> Se muestra la matriz a predecir:\n")
print(y)

>> Se muestra la matriz categorica:

[[1.47000000e+02 0.00000000e+00 4.11764706e+01 0.00000000e+00
  0.00000000e+00]
 [8.90000000e+02 1.00000000e+00 3.48200313e+01 0.00000000e+00
  0.00000000e+00]
 [4.60000000e+01 0.00000000e+00 4.38095238e+01 0.00000000e+00
  0.00000000e+00]
 [8.98000000e+02 0.00000000e+00 3.50097466e+01 0.00000000e+00
  0.00000000e+00]
 [8.20000000e+02 0.00000000e+00 3.49531117e+01 0.00000000e+00
  0.00000000e+00]
 [1.94000000e+02 0.00000000e+00 2.81976744e+01 0.00000000e+00
  0.00000000e+00]
 [8.99000000e+02 0.00000000e+00 3.52549020e+01 0.00000000e+00
  0.00000000e+00]
 [3.38700000e+03 0.00000000e+00 3.51239241e+01 0.00000000e+00
  0.00000000e+00]
 [1.24000000e+02 0.00000000e+00 4.49275362e+01 0.00000000e+00
  0.00000000e+00]
 [1.27000000e+02 0.00000000e+00 4.60144928e+01 0.00000000e+00
  0.00000000e+00]
 [9.39000000e+02 0.00000000e+00 3.46878463e+01 0.00000000e+00
  0.00000000e+00]
 [3.44000000e+03 0.00000000e+00 3.55005160e+01 0.00000000e+00
  0.00000000e+00]
 [8

Se separan los datos con el 70% de los datos para entrenamiento y el 30% para testing:

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)

Mediante el constructor inicializa la instancia_red.

In [12]:
instancia_red = MLPClassifier(solver='adam', random_state=0, max_iter = 2000)
print(instancia_red)
instancia_red.fit(X_train,y_train)

MLPClassifier(max_iter=2000, random_state=0)


MLPClassifier(max_iter=2000, random_state=0)

Las predicciones del testing se muestran:

In [13]:
print(Back.WHITE + " >>> PREDICCIONES DEL TESTING <<< \n\n" + Back.RESET + str(instancia_red.predict(X_test)))

 >>> PREDICCIONES DEL TESTING <<< 

['R5' 'X4' 'X4' 'R5' 'R5' 'R5' 'R5' 'R5' 'R5' 'R5' 'R5' 'R5' 'R5' 'R5'
 'X4' 'R5' 'R5' 'R5' 'X4' 'R5' 'R5' 'X4' 'R5' 'R5' 'X4' 'R5' 'R5' 'R5'
 'R5' 'X4' 'R5' 'R5' 'R5' 'R5' 'R5' 'X4']


## ▪️ Índices de calidad del modelo.

In [14]:
prediccion = instancia_red.predict(X_test)
MC = confusion_matrix(y_test, prediccion)
indices = indices_general(MC,list(np.unique(y)))
print(Back.YELLOW + "\n - Índices obtenidos: " + Back.RESET + Fore.RESET)
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


 - Índices obtenidos: 

> Matriz de Confusión:
[[25  0]
 [ 3  8]]

> Precisión Global:
0.9166666666666666

> Error Global:
0.08333333333333337


## ▪️ Predicción de tropismos R5X4.

In [15]:
# Se lee el archivo de secuencias.
data = pd.read_excel('NumeroAccesoSecuencias.xlsx')
print(Back.WHITE  + " >>> PREDICCION PARA TROPISMOS R5X4: <<< \n" + Back.RESET + Fore.RESET)

# Se recorren todos los identificadores "R5X4".
for iden in data['R5X4'].dropna():
    fila_aux = []
    
    # Se obtienen los datos del identificador y se agregan al data frame.
    fila = getIdenData(iden)
    for i in col:
        fila_aux.append(fila[i])
        
    # Se realiza la predicción y se imprime el resultado.
    prediccion = instancia_red.predict(np.array([fila_aux]).reshape(1, -1))
    print(Fore.LIGHTBLUE_EX + "\t" + iden + Fore.RESET +  "\tCorreceptor: " + Fore.LIGHTGREEN_EX + str(prediccion))
    
print(Fore.GREEN  + "\n\n+++ PROCESO COMPLETADO +++" + Fore.RESET)

 >>> PREDICCION PARA TROPISMOS R5X4: <<< 

	AB014795	Correceptor: ['X4']
	AF062029	Correceptor: ['X4']
	AF062031	Correceptor: ['X4']
	AF062033	Correceptor: ['X4']
	AF107771	Correceptor: ['R5']
	U08680	Correceptor: ['R5']
	U08682	Correceptor: ['R5']
	U08444	Correceptor: ['R5']
	U08445	Correceptor: ['R5']
	AF355674	Correceptor: ['X4']
	AF355647	Correceptor: ['X4']
	AF355630	Correceptor: ['X4']
	AF355690	Correceptor: ['X4']
	M91819	Correceptor: ['R5']
	AF035532	Correceptor: ['R5']
	AF035533	Correceptor: ['R5']
	AF259019	Correceptor: ['X4']
	AF259025	Correceptor: ['X4']
	AF259021	Correceptor: ['X4']
	AF259041	Correceptor: ['X4']
	AF258970	Correceptor: ['X4']
	AF258978	Correceptor: ['X4']
	AF021607	Correceptor: ['R5']
	AF204137	Correceptor: ['X4']
	AF112925	Correceptor: ['R5']
	M17451	Correceptor: ['R5']
	K02007	Correceptor: ['R5']
	U39362	Correceptor: ['R5']
	AF069140	Correceptor: ['R5']
	AF458235	Correceptor: ['X4']
	AF005494	Correceptor: ['R5']


+++ PROCESO COMPLETADO +++
